In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
'''
this code is run in Colab Google. So, this code takes folder from drive.
So if you run it in your pc you need not run the code to mount the drive.

And all the folder links are given with respect to the drive. So , if you want to run this code in your machine please 

change the path accrodingly.

For example :

if drive path is : '/content/gdrive/My Drive/train/fold_0/all/*.bmp'
then for your machine it will be  : 'YOUR_FOLDER_NAME/train/fold_0/all/*.bmp'

'''


from keras import applications
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Flatten,Dropout,Conv2D,MaxPooling2D
from keras.models import Sequential
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import glob
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))

/home/ashiq/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<Figure size 2000x1000 with 0 Axes>

In [0]:
from scipy.signal import convolve2d
def plti(im, h=8, **kwargs):
    """
    Helper function to plot an image.
    """
    y = im.shape[0]
    x = im.shape[1]
    w = (y/x) * h
    plt.figure(figsize=(w,h))
    plt.imshow(im, interpolation="none", **kwargs)
    plt.axis('off')

def edge(im_small):
  n=100
  sobel_x = np.c_[
      [-1,0,1],
      [-2,0,2],
      [-1,0,1]
  ]

  sobel_y = np.c_[
      [1,2,1],
      [0,0,0],
      [-1,-2,-1]
  ]

  ims = []
  for d in range(3):
      sx = convolve2d(im_small[:,:,d], sobel_x, mode="same", boundary="symm")
      sy = convolve2d(im_small[:,:,d], sobel_y, mode="same", boundary="symm")
      ims.append(np.sqrt(sx*sx + sy*sy))

  im_conv = np.stack(ims, axis=2).astype("uint8")
  return im_conv

def emboss(im_small):
  karnel = np.array(
  [
      [-1,-1,0],
      [-1,0,1],
      [0,1,1]
  ])
  ims = []
  for d in range(3):
      sx = convolve2d(im_small[:,:,d], karnel, mode="same", boundary="symm")
      ims.append(sx)
  im_conv = np.stack(ims, axis=2).astype("uint8")
  return im_conv

def channel_diff(im):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(20,20))

    for c, ax in zip(range(3), axs):
        tmp_im = np.zeros(im.shape, dtype="uint8")
        tmp_im[:,:,c] = im[:,:,c]
        ax.imshow(tmp_im)
        ax.set_axis_off()

def to_grayscale(im, weights = np.c_[0.2989, 0.5870, 0.1140]):
    """
    Transforms a colour image to a greyscale image by
    taking the mean of the RGB values, weighted
    by the matrix weights
    """
    tile = np.tile(weights, reps=(im.shape[0],im.shape[1],1))
    img= np.sum(tile * im, axis=2)
    return img
    #plti(img, cmap='Greys')

In [0]:
Images = []
labels = []
dim = 128
from scipy import ndimage
alpha = 1
#print(glob.glob('/content/gdrive/My Drive/train/fold_1/all/*.bmp'))
m=0
number = 1000
for filename in glob.glob('/content/gdrive/My Drive/train/fold_0/all/*.bmp'): #assuming gif
    m+=1
    if m>number:
      break
    im = image.load_img(filename, target_size=(dim, dim))
    im = image.img_to_array(im)
    im = np.array(im)
    
    filter_blurred_f = ndimage.gaussian_filter(im, 1)
    sharpened = im + alpha * (im - filter_blurred_f)
    
    #im = edge(im)
    #im = emboss(im)
    #im = edge(im)
    Images.append(im)
    labels.append(1)
    print(m)
m=0
for filename in glob.glob('/content/gdrive/My Drive/train/fold_1/all/*.bmp'): #assuming gif
    m+=1
    if m>10:
      break
    try:
      im = image.load_img(filename, target_size=(dim, dim))
      im = image.img_to_array(im)
      im = np.array(im)
      filter_blurred_f = ndimage.gaussian_filter(im, 1)
      sharpened = im + alpha * (im - filter_blurred_f)
      
      #im = edge(im)
      #im = emboss(im)
      #im = edge(im)
      Images.append(im)
      labels.append(1)
    except(OSError):
      print('error',m)
print(m)
m=0
for filename in glob.glob('/content/gdrive/My Drive/train/fold_2/all/*.bmp'): #assuming gif
    m+=1
    if m>number:
      break
    im = image.load_img(filename, target_size=(dim, dim))
    im = image.img_to_array(im)
    im = np.array(im)
    filter_blurred_f = ndimage.gaussian_filter(im, 1)
    sharpened = im + alpha * (im - filter_blurred_f)
    
    #im = edge(im)
    #im = emboss(im)
    #im = edge(im)
    Images.append(im)
    labels.append(1)
print(m)
m=0    
for filename in glob.glob('/content/gdrive/My Drive/train/fold_0/hem/*.bmp'): #assuming gif
    m+=1
    if m>number:
      break
    im = image.load_img(filename, target_size=(dim, dim))
    im = image.img_to_array(im)
    im = np.array(im)
    filter_blurred_f = ndimage.gaussian_filter(im, 1)
    sharpened = im + alpha * (im - filter_blurred_f)
    
    #im = edge(im)
    #im = emboss(im)
    #im = edge(im)
    Images.append(im)
    labels.append(0)
print(m)
m=0
for filename in glob.glob('/content/gdrive/My Drive/train/fold_1/hem/*.bmp'): #assuming gif
    m+=1
    if m>number:
      break
    im = image.load_img(filename, target_size=(dim, dim))
    im = image.img_to_array(im)
    im = np.array(im)
    filter_blurred_f = ndimage.gaussian_filter(im, 1)
    sharpened = im + alpha * (im - filter_blurred_f)
    
    #im = edge(im)
    #im = emboss(im)
    #im = edge(im)
    Images.append(im)
    labels.append(0)
print(m)
m=0
for filename in glob.glob('/content/gdrive/My Drive/train/fold_2/hem/*.bmp'): #assuming gif
    m+=1
    if m>number:
      break
    im = image.load_img(filename, target_size=(dim, dim))
    im = image.img_to_array(im)
    im = np.array(im)
    filter_blurred_f = ndimage.gaussian_filter(im, 1)
    sharpened = im + alpha * (im - filter_blurred_f)
   
    #im = edge(im)
    #im = emboss(im)
    #im = edge(im)
    Images.append(im)
    labels.append(0)
print(m)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
print(len(Images), Images[0].shape)

i = Images[19]
plt.imshow( image.array_to_img(Images[40]) )
channel_diff(i)
#to_grayscale(i)
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(50,50))
axs[0].imshow(emboss(i))
axs[1].imshow(edge(i))
axs[2].imshow(edge(emboss(i)), interpolation="none", cmap="Greys")
axs[3].imshow(emboss(edge(i)), interpolation="none", cmap="Greys")
              
i = Images[49]
#plt.imshow( image.array_to_img(Images[40]) )
channel_diff(i)
#to_grayscale(i)
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(50,50))
axs[0].imshow(emboss(i))
axs[1].imshow(edge(i))
axs[2].imshow(edge(emboss(i)), interpolation="none", cmap="Greys")
axs[3].imshow(emboss(edge(i)), interpolation="none", cmap="Greys")
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(50,50))
axs[0].imshow( to_grayscale(emboss(i)))
axs[1].imshow( to_grayscale(edge(i)))
axs[2].imshow( to_grayscale(edge(emboss(i))), interpolation="none", cmap="Greys")
axs[3].imshow( to_grayscale(emboss(edge(i))), interpolation="none", cmap="Greys")


In [0]:
from keras.applications.vgg16 import VGG16
from keras.applications.nasnet import NASNetMobile
from keras.applications.mobilenet_v2 import MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False,input_shape=(dim,dim,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
# let's add a fully-connected layer
x= Dropout(0.25)(x)
x = Dense(100, activation='relu')(x)
x= Dropout(0.25)(x)
#x = Dense(2, activation='softmax')
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, activation='sigmoid')(x)


model = Model(inputs=base_model.input, outputs=predictions)

for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


print(len(model.layers))
for layer in model.layers:
   layer.trainable = True


# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [0]:
def getModels(dimension,channel):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(dimension,dimension,channel)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(GlobalAveragePooling2D())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='softmax'))

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model
def getModels_2(dimension,channel):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(dimension,dimension,channel)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(GlobalAveragePooling2D())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='softmax'))

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model
  

In [0]:
import numpy as np
from keras.utils import np_utils

indices=list(range(len(Images)))
np.random.seed(23)
np.random.shuffle(indices)
Images = np.array(Images)
Images= Images/255.0
labels = np.array(labels)
#labels=np_utils.to_categorical(labels)
ind=int(len(indices)*0.60)
ind2 = int( len(indices)*0.20)
# train data
X_train=Images[indices[:ind]] 
y_train=labels[indices[:ind]]
# validation data
X_val=Images[indices[ind:ind+ind2]] 
y_val=labels[indices[ind:ind+ind2]]

x_test = Images[indices[ind+ind2:]]
y_test=labels[indices[ind+ind2:]]

In [0]:
print(X_train[0].shape)

In [11]:
#model = getModels(200,3)
from keras.callbacks import ModelCheckpoint
path_model='model_simple_keras_starter.h5' # save model at this location after each epoch
checkpointer = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)
model.fit(x=X_train, y=y_train,
            epochs=40, 
            verbose=1, 
            validation_data=(X_val,y_val),
            shuffle=True,
          callbacks=[
                checkpointer,
            ]
            
            )

Train on 2989 samples, validate on 996 samples
Epoch 1/40
2989/2989 [==============================] - 605s 202ms/step - loss: 0.4837 - acc: 0.7819 - val_loss: 0.4658 - val_acc: 0.7861

Epoch 00001: val_loss improved from inf to 0.46579, saving model to model.h5
Epoch 2/40
1728/2989 [================>.............] - ETA: 3:52 - loss: 0.3428 - acc: 0.8513

KeyboardInterrupt: ignored

In [12]:
print(model.evaluate(x_test,y_test))

998/998 [==============================] - 45s 45ms/step
[0.6327514168255792, 0.8036072145483059]
